# Agents for Amazon Bedrock - Testing Agent Invocation

This notebook provides sample code for testing the agent invocation using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html) client for `bedrock-agent-runtime`

### Use Case
Now that we've created the agent and attached the knowledge base to it, we will use the runtime client to invoke the agent with different queries. The boto3 sdk for agent is divided into two clients: `bedrock-agent` and `bedrock-agent-runtime`. The `bedrock-agent` client is responsible for the functionalities to create, update, delete and/or prepare an agent or a knowledge base. While the `bedrock-agent-runtime` is responsible for invoke an agent (with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API) and retrieve documents from a knowledge base (with the [`retrieve`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [`retrieve_and_generate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) APIs). This notebook will focus on the runtime invocation of the agent created in the previous notebooks. We will use the `invoke_agent` API.



### Notebook Walkthrough

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Invoke agent with a new session using Knowledge Bases and Action Groups
- Invoke agent with an existent session
- Invoke agent with prompt attributes
- Invoke agent with trace enabled
- Invoke agent with different languages for the request


### Next Steps: 
In the next lab, we will clean up the resources created

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [2]:
%store -r

#### Importing required packages and initiate variables

Let's also import the necessary packages, setup the logging and initiate the boto3 client for `bedrock-agent-runtime`. We are also initiating the client for `dynamodb` as a support functionality to check that the tasks were performed correctly by our agent

In [3]:
import boto3
import logging
import pprint
import json
import pandas as pd
from agent import invoke_agent_helper
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
dynamodb = boto3.resource('dynamodb')

#### Create support invokeAgent function

Let's again use the same support function called `invoke_agent_helper` from `agents.py` to allow us to invoke the agent with or without trace enabled and with or without session state. 

This function allows the user to send a `query` to the agent with a `session_id`. The user can then decide to enable trace or not using the `enable_trace` boolean variable and to pass a session state as a dictionary via the `session_state` variable.

If a new `session_id` is provided, the agent will create a new conversation without previous context. If the same `session_id` is reused, the conversation history related to that session id is made available to the agent.

If the `enable_trace` is set to `True`, each response from the agent is accompanied by a *trace* that details the step being orchestrated by the agent. It allows you to follow the agent's (reasoning via Chain of Thoughts prompting) that led to the final response at that point of the conversation.

Finally, you can also pass a session context using the `session_state` parameter. The session state allows you to share the following information with the agent:
- **`sessionAttributes`**: attributes that persist over a session between the user and the agent. All invokeAgent calls with the same session_id belong to the same sesison and will have the sessionAttributes shared with them as long as the session time limit has not being surpassed and the user has not ended the session. The sessionAttributes are available in the lambda function but are **not** added to the agent's prompt. As a result, you can only use session attributes if your lambda function can handle them. You can find more examples of using a session attribute [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/06-prompt-and-session-attributes). It is also a good pattern to implement fine-grained access control for certain APIs using the lambda function integration. You can find an example for it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/09-fine-grained-access-permissions)
- **`promptSessionAttributes`**: attributes that persist over a single invokeAgent call. Prompt attributes are added to the prompt and to the lambda function. You can also use the `$prompt_session_attributes$` placeholder when editing the orchestration base prompt.
- **`invocationId`**: The id returned by the agent in the [ReturnControlPayload](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_ReturnControlPayload.html) object in the returnControl field of the InvokeAgent response. This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).
- **`returnControlInvocationResults`**: the results obtained from invoking the action outside of agents for Amazon Bedrock.  This field is required if passing the answer of a Return of Control invocation. You can find an example of how to use it [here](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/agents-for-bedrock/features-examples/03-create-agent-with-return-of-control).

#### Create support selectAllFromDynamoDB function

We will also create the support function called `selectAllFromDynamoDB` to select all data in the dynamoDB table `restaurant_bookings`. This function will be used to validate our agent's behaviour

In [4]:
def selectAllFromDynamodb():
    # Get the table object
    table = dynamodb.Table(table_name)

    # Scan the table and get all items
    response = table.scan()
    items = response['Items']

    # Handle pagination if necessary
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        items.extend(response['Items'])

    items = pd.DataFrame(items)
    return items

In [5]:
# test function invocation
items = selectAllFromDynamodb()
items

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,199c52f5,Anna


### Invoking agent with a new session id
Let's first use the `InvokeAgent` function to query the Knowledge Base with the agent without previous context

In [6]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What is in the childrens menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The children's menu at The Regrettable Experience includes the following items:

Entrees:
1. Chicken nuggets (allergens: gluten, possible soy)
2. Macaroni and cheese (allergens: dairy, gluten)
3. Mini cheese quesadillas (allergens: dairy, gluten)
4. Peanut butter and banana sandwich (allergens: nuts, gluten)
5. Veggie pita pockets (allergens: gluten, possible soy)

Mains:
1. Mini cheeseburgers (allergens: dairy, gluten)
2. Fish sticks (allergens: gluten, possible soy)
3. Grilled cheese sandwich (allergens: dairy, gluten)
4. Spaghetti with marinara sauce (allergen: gluten)
5. Mini pita pizza (allergens: dairy, gluten)

Desserts:
1. Mini ice cream sundae (allergen: dairy)
2. Fruit kabobs (no allergens)
3. Chocolate chip cookie bites (allergens: dairy, gluten)
4. Banana split (allergen: dairy)
5. Jello cups (no allergens)
<sources>45</sources>
CPU times: user 19.2 ms, sys: 1.81 ms, total: 21 ms
Wall time: 33.9 s


### Invoking agent with existent session id

Next we can use the context to ask a follow up question. To do so, we use the same `session_id` with the `invokeAgent` function

In [7]:
%%time
query = "Which of those options are vegetarian?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

To determine which children's menu items are vegetarian, I will need to review the list and identify any items that do not contain meat or meat-derived ingredients.


Based on the children's menu items listed, the following options appear to be vegetarian:

Entrees:
2. Macaroni and cheese (allergens: dairy, gluten)
3. Mini cheese quesadillas (allergens: dairy, gluten)
5. Veggie pita pockets (allergens: gluten, possible soy)

Mains: 
3. Grilled cheese sandwich (allergens: dairy, gluten)
4. Spaghetti with marinara sauce (allergen: gluten)
5. Mini pita pizza (allergens: dairy, gluten)

Desserts:
2. Fruit kabobs (no allergens)
3. Chocolate chip cookie bites (allergens: dairy, gluten)
5. Jello cups (no allergens)

<sources>45</sources>

CPU times: user 21.5 ms, sys: 2.31 ms, total: 23.9 ms
Wall time: 9.99 s


As you can see, the agent knows that we are talking about the children's menu.

### Invoking agent using action group

Now let's use our agent to make a reservation. By doing so, we will require the agent to execute an action from our action group to create a new reservation.

In [8]:
%%time
query = "Hi, I am Maria. I want to create a booking for 4 people, at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Thank you Maria, your reservation for 4 people on May 5th, 2024 at 9pm has been created successfully. Your booking ID is c5a41a81. Please let me know if you need any other assistance with your reservation.
CPU times: user 18 ms, sys: 3.11 ms, total: 21.1 ms
Wall time: 16.2 s


In [9]:
%%time
query = "Hi, I am Maria. I want to create a booking for 4 people, at 9pm on the 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Maria, you already have an existing reservation booked for 4 people on May 5th, 2024 at 9pm with the booking ID c5a41a81. There is no need to create a new booking since you have one scheduled for the same date, time and number of guests. Please let me know if you need to modify or cancel your existing reservation instead.
CPU times: user 18.9 ms, sys: 1.64 ms, total: 20.6 ms
Wall time: 18.2 s


Let's double check that the data was properly added to the dynamoDB table

In [10]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,199c52f5,Anna
1,4,2024-05-05,21:00,c5a41a81,Maria


In [11]:
%%time
query = "how many reservations have been made for 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Unfortunately, I do not have a way to directly query how many reservations have been made for a specific date with the functions provided to me. The <REDACTED> function only allows retrieving details for an individual booking ID. Without access to documentation or additional functions to query bookings by date, I cannot determine the number of reservations made for May 5th, 2024. I apologize that I cannot provide that information.
CPU times: user 20.3 ms, sys: 5.12 ms, total: 25.4 ms
Wall time: 22 s


In [12]:
%%time
query = "how many reservations have been made for 5th of May 2024."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

I apologize, but I do not have the capability to query or retrieve the total number of reservations made for a specific date like May 5th, 2024. The functions provided to me only allow getting details of an individual booking by its ID or creating/deleting a new booking. Without access to additional data querying functions or documentation about the booking system, I cannot determine how many total reservations exist for that date. I'm afraid I do not have enough information or permissions to answer your question accurately.
CPU times: user 24.9 ms, sys: 0 ns, total: 24.9 ms
Wall time: 12.1 s


Great! We've used our agent to do create a reservation. However, often when booking tables in restaurants we are already logged in to a system that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [promptSessionAttributes](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [13]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 6th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking for 2 guests on May 6th, 2024 at 8pm under the name John has been created successfully. The booking ID is 73388164.
CPU times: user 22.7 ms, sys: 0 ns, total: 22.7 ms
Wall time: 14.6 s


Again let's validate the the correct data was added to dynamoDB

In [14]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,199c52f5,Anna
1,4,2024-05-05,21:00,c5a41a81,Maria
2,2,2024-05-06,20:00,73388164,John


We can also validate the data using our agent's session information since the agent knows the booking id to invoke the get booking details function

In [18]:
%%time
query = "Get the details for the booking created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Here are the details for the booking with ID 73388164:

Name: John
Date: 2024-05-06
Time: 20:00 (8pm)
Number of Guests: 2
CPU times: user 20.4 ms, sys: 294 µs, total: 20.7 ms
Wall time: 8.52 s


In [19]:
%%time
query = "Get the details for all bookings created"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Unfortunately, I do not have the capability to retrieve details for all bookings that have been created. The functions provided to me only allow getting details for a specific booking by providing the booking ID, creating a new booking, or deleting an existing booking. I apologize that I cannot provide the information you requested about all bookings.
CPU times: user 14.6 ms, sys: 4.46 ms, total: 19.1 ms
Wall time: 7.85 s


#### Deleting booking created

Let's also test the delete booking functionality by deleting the last created booking id

In [20]:
%%time
query = f"I want to delete the booking."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The booking with ID 73388164 has been deleted successfully.
CPU times: user 18.2 ms, sys: 4.43 ms, total: 22.6 ms
Wall time: 7.6 s


And we confirm that the booking has also been deleted from the dynamoDB table

In [22]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-05-05,20:00,199c52f5,Anna
1,4,2024-05-05,21:00,c5a41a81,Maria


#### Invoking agent using promptSessionAttributes to handle temporal information

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Agents for Amazon Bedrock also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [23]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

'Jun-26-2024'

In [24]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John",
        "today": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Your booking for 2 people at 8pm on June 27th, 2024 under the name John has been created successfully. The booking ID is 01ecc61f.
CPU times: user 19.4 ms, sys: 642 µs, total: 20.1 ms
Wall time: 18.5 s


And now to confirm that everything got added properly, let's retrieve the details of the last booking 

In [25]:
%%time
query = "Could you get the details for the last booking created?"
booking_id = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(booking_id)

Here are the details for the booking with ID 01ecc61f:

Name: John
Number of Guests: 2
Date: June 27th, 2024
Time: 8:00 PM
CPU times: user 16.6 ms, sys: 2.43 ms, total: 19 ms
Wall time: 9.92 s


#### Asking the agent for food recomendations

Another good use case for our agent, is to use its reasoning capabilities to ask for some food recommendation. Let's look at a couple of examples for it

In [27]:
%%time
session_id:str = str(uuid.uuid1())
query = "What do you have for kids that don't like fries?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


For kids who don't like french fries, some good menu options are:
<sources>
4
5
</sources>



1. Fruit Kabobs - Fresh fruit chunks on skewers for a healthy and fun side dish
<sources>
4
</sources>



2. Veggie Pita Pockets - Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes
<sources>
5
</sources>



3. Jello Cups - Colorful and wiggly gelatin cups
<sources>
4
</sources>

CPU times: user 23 ms, sys: 6.53 ms, total: 29.6 ms
Wall time: 30 s


In [28]:
%%time
session_id:str = str(uuid.uuid1())
query = "I am allergic to shrimps. What can I eat at this restaurant?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)


Based on the menu information, here are some dishes that do not contain shrimp that you can order at this restaurant:

- Buffalo Chicken Wings
- Caprese Salad Stuffed Avocado  
- Loaded Potato Skins
- Vegetarian Chili
- Grilled BBQ Chicken
- Southern Fried Catfish
- BBQ Pulled Pork Sandwiches
- Chicken Nuggets
- Macaroni and Cheese
- Mini Cheese Quesadillas
- Peanut Butter and Banana Sandwich
- Veggie Pita Pockets
- Mini Cheeseburgers
- Fish Sticks
- Grilled Cheese Sandwich
- Spaghetti with Marinara Sauce

<sources>
5
3
</sources>

CPU times: user 15.8 ms, sys: 4.02 ms, total: 19.8 ms
Wall time: 31.7 s


### Invoking agent with trace enabled

We can also invoke our agent with the trace enabled to produce the details of each step being orchestrated by the Agent. Let's see all the steps executed when checking for documents in the knowledge base

In [29]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=True)
print(response)

[2024-06-26 17:52:59,176] p267 {agent.py:170} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 26 Jun 2024 17:52:59 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ecd8b508-33e4-11ef-b9fc-cabdcde3fd2b',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '45a6c5e5-9747-4c36-bb9c-f38c519478ce'},
                      'HTTPStatusCode': 200,
                      'RequestId': '45a6c5e5-9747-4c36-bb9c-f38c519478ce',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd540e0eec0>,
 'contentType': 'application/json',
 'sessionId': 'ecd8b508-33e4-11ef-b9fc-cabdcde3fd2b'}


[2024-06-26 17:52:59,385] p267 {agent.py:184} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "O096M3WGHH",
  "agentVersion": "DRAFT",
  "sessionId": "ecd8b508-33e4-11ef-b9fc-cabdcde3fd2b",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\"        You are a restaurant agent, helping clients retrieve information from their booking,create a new booking or delete an existing booking        You have been provided with a set of functions to answer the user's question.        You must call the functions in the format below:        <function_calls>        <invoke>            <tool_name>$TOOL_NAME</tool_name>            <parameters>            <$PARAMETER_NAME>$PARAMETE

Based on the provided menu information, the desserts offered on the adult dinner menu at The Regrettable Experience restaurant are:

1. Classic New York Cheesecake - Creamy cheesecake with a graham cracker crust, topped with a choice of fruit compote or chocolate ganache. Allergens: Dairy, Gluten.

2. Apple Pie A la Mode - Warm apple pie with a flaky crust, served with a scoop of vanilla ice cream and a drizzle of caramel sauce. Allergens: Dairy, Gluten.

3. Chocolate Lava Cake - Rich and gooey chocolate cake with a molten center, dusted with powdered sugar and served with a scoop of raspberry sorbet. Allergens: Dairy, possible Gluten and Soy.

4. Pecan Pie Bars - Buttery shortbread crust topped with a gooey pecan filling, cut into bars for easy serving. Allergens: Dairy, Nuts, Gluten.

5. Banana Pudding Parfait - Layers of vanilla pudding, sliced bananas, and vanilla wafers, topped with whipped cream and a sprinkle of crushed nuts. Allergens: Dairy, Gluten.<sources>5</sources>
CPU tim

### Invoking agent with different prompt languages

The last feature that we will highlight in this notebook is the ability of LLM models to handle inputs in different languages. We've only implemented our agent in English, but wouldn't it be great if it could also handle other languages?

Good news is that it can! This is due to the LLM multi-language capabilities. And the best part is that we don't need to change anything in the agent and it will even going to respond in the requested language.

Let's try a couple of queries!

In [30]:
%%time
# Invoking agents in spanish
session_id:str = str(uuid.uuid1())
query = "¿Podrías reservar una mesa para dos 25/07/2024 a las 19:30"
session_state = {
    "promptSessionAttributes": {
        "Nombre": "Gabriela"
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

He creado una reserva para 2 personas a nombre de Gabriela el 25 de julio de 2024 a las 19:30 horas. El número de reserva es 26db209b.
CPU times: user 20.8 ms, sys: 0 ns, total: 20.8 ms
Wall time: 8.91 s


In [31]:
%%time
# Invoking agents in german
session_id:str = str(uuid.uuid1())
query = "Könnten Sie heute Abend einen Tisch für zwei reservieren? Um 19:30 Uhr"
session_state = {
    "promptSessionAttributes": {
        "Name": "Julian",
        "Heute": today
    }
}
response = invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=session_state)
print(response)

Ihre Reservierung für heute Abend um 19:30 Uhr für 2 Personen unter dem Namen Julian wurde erfolgreich vorgenommen. Ihre Reservierungsnummer ist 98d9153e.
CPU times: user 17.6 ms, sys: 3.18 ms, total: 20.8 ms
Wall time: 16.8 s


Last, let's check our dynamoDB to see all the bookings available

In [32]:
selectAllFromDynamodb()

,num_guests,date,hour,booking_id,name
0,2,2024-06-26,19:30,98d9153e,Julian
1,2,2024-06-27,20:00,01ecc61f,John
2,2,2024-05-05,20:00,199c52f5,Anna
3,2,2024-07-25,19:30,26db209b,Gabriela
4,4,2024-05-05,21:00,c5a41a81,Maria


### Next Steps

Next we will delete all the resources created